In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn import metrics
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import AdaBoostRegressor,VotingRegressor,RandomForestRegressor,GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# for 6 hour Model_2

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-1)
df['one_day_fog_index'] = df['fog_index'].shift(3)

#Adding more columns
df['two_day_fog_index'] = df['one_day_fog_index'].shift(4)
df['three_day_fog_index'] = df['two_day_fog_index'].shift(4)
df['four_day_fog_index'] = df['three_day_fog_index'].shift(4)
df['five_day_fog_index'] = df['four_day_fog_index'].shift(4)

df.dropna(inplace=True)

In [ ]:
df.head(20)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index,fog_index_future,one_day_fog_index,two_day_fog_index,three_day_fog_index,four_day_fog_index,five_day_fog_index
date_time,,,,,,,,,,,,,,,
2000-01-14 04:00:00,11.281818,11.218182,99.591818,101086.820036,0.521572,6627.893282,21600.0,1,0.306847,0.125859,0.084252,0.000000,0.000000,0.000000,0.540086
2000-01-14 10:00:00,15.558333,12.533333,82.696250,101347.597725,2.966550,906.186258,7200.0,1,0.125859,0.088994,0.000000,0.000000,0.082672,0.000000,0.407318
2000-01-14 16:00:00,21.916667,11.472222,53.096574,101139.240678,3.352792,160.189388,1800.0,1,0.088994,0.000000,0.087940,0.000000,0.000000,0.000000,0.108783
2000-01-14 22:00:00,14.016667,11.016667,82.150833,101224.841083,2.481066,0.000000,0.0,1,0.000000,0.088994,0.306847,0.088994,0.000000,0.000000,0.540086
2000-01-15 04:00:00,12.483333,10.241667,86.192778,101137.359350,2.229606,160.189388,1800.0,1,0.088994,0.082739,0.125859,0.084252,0.000000,0.000000,0.000000
2000-01-15 10:00:00,15.295455,10.722727,74.167500,101403.909282,2.377434,446.788689,5400.0,1,0.082739,0.000000,0.088994,0.000000,0.000000,0.082672,0.000000
2000-01-15 16:00:00,21.163636,8.645455,45.156364,101206.882955,3.145528,0.000000,0.0,1,0.000000,0.000000,0.000000,0.087940,0.000000,0.000000,0.000000
2000-01-15 22:00:00,13.597222,8.194444,70.075694,101460.477392,3.486903,0.000000,0.0,1,0.000000,0.000000,0.088994,0.306847,0.088994,0.000000,0.000000
2000-01-16 04:00:00,9.159259,4.707407,73.827963,101674.478422,2.816345,0.000000,0.0,1,0.000000,0.000000,0.082739,0.125859,0.084252,0.000000,0.000000


In [ ]:


# selecting winter month(NDJF)
#train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]

#train.dropna(inplace=True)
X_train,y_train = df.drop(["fog_index_future"],axis=1),df["fog_index_future"]


# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-1)
val['one_day_fog_index'] = val['fog_index'].shift(3)

#Adding more columns
val['two_day_fog_index'] = val['one_day_fog_index'].shift(4)
val['three_day_fog_index'] = val['two_day_fog_index'].shift(4)
val['four_day_fog_index'] = val['three_day_fog_index'].shift(4)
val['five_day_fog_index'] = val['four_day_fog_index'].shift(4)

val.dropna(inplace=True)

# selecting winter month(NDJF)
#validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]

#validate.dropna(inplace=True)
X_val,y_val = val.drop(["fog_index_future"],axis=1),val["fog_index_future"]


# taking test dataset
test = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/test_preprocessed.csv')

test["date_time"]=pd.to_datetime(test.date_time)
test.set_index("date_time",inplace=True)

test['fog_index_future']= test['fog_index'].shift(-1)
test['one_day_fog_index'] = test['fog_index'].shift(3)

#Adding more columns
test['two_day_fog_index'] = test['one_day_fog_index'].shift(4)
test['three_day_fog_index'] = test['two_day_fog_index'].shift(4)
test['four_day_fog_index'] = test['three_day_fog_index'].shift(4)
test['five_day_fog_index'] = test['four_day_fog_index'].shift(4)

test.dropna(inplace=True)

# selecting winter month(NDJF)
#test_data =  test[(test.index.month==1) | (test.index.month==2)|(test.index.month==11) |(test.index.month==12)]

#test_data.dropna(inplace=True)
X_test,y_test = test.drop(["fog_index_future"],axis=1),test["fog_index_future"]


stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)
X_test_stand = stand_scaler.transform(X_test)

In [ ]:
X_train.isna().sum()

avg_air_temp             0
avg_dew_point            0
avg_relative_humidity    0
avg_pressure             0
avg_visibility           0
energy_loss              0
fog_duration             0
fog_month                0
fog_index                0
one_day_fog_index        0
two_day_fog_index        0
three_day_fog_index      0
four_day_fog_index       0
five_day_fog_index       0
dtype: int64

In [ ]:
lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))
print(metrics.mean_squared_error(y_test,lr.predict(X_test_stand)))

LinearRegressor
0.012485396778283801
5.720494325796902
5.740634024571515


In [ ]:
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))
print(metrics.mean_squared_error(y_test,rf.predict(X_test)))

RandomforestRegressor
0.0015134642907060948
0.018122742452957738
0.015856821206983614


In [ ]:
# lr.coef_

In [ ]:
# X_train.head(2)

In [ ]:
df.head(30)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index,fog_index_future,one_day_fog_index,two_day_fog_index,three_day_fog_index,four_day_fog_index,five_day_fog_index
date_time,,,,,,,,,,,,,,,
2000-01-01 04:00:00,10.000000,10.000000,100.000000,101631.207878,0.193121,10693.704233,19800.0,1,0.540086,0.407318,NaN,NaN,NaN,NaN,NaN
2000-01-01 10:00:00,10.860000,10.430000,97.136333,101849.065640,0.410382,8798.058547,21600.0,1,0.407318,0.108783,NaN,NaN,NaN,NaN,NaN
2000-01-01 16:00:00,13.175000,11.625000,90.555000,101629.796892,1.425607,2349.704586,21600.0,1,0.108783,0.540086,NaN,NaN,NaN,NaN,NaN
2000-01-01 22:00:00,9.250000,9.125000,98.656250,101722.922625,0.193121,11665.859163,21600.0,1,0.540086,0.000000,0.540086,NaN,NaN,NaN,NaN
2000-01-02 04:00:00,14.304545,10.927273,80.156818,101076.814791,2.980937,0.000000,0.0,1,0.000000,0.000000,0.407318,NaN,NaN,NaN,NaN
2000-01-02 10:00:00,17.990000,11.290000,65.721000,101060.036770,3.862416,0.000000,0.0,1,0.000000,0.000000,0.108783,NaN,NaN,NaN,NaN
2000-01-02 16:00:00,22.090000,12.240000,54.080000,100782.352800,3.822182,0.000000,0.0,1,0.000000,0.000000,0.540086,NaN,NaN,NaN,NaN
2000-01-02 22:00:00,16.550000,13.590000,82.901500,101024.479680,3.661249,0.000000,0.0,1,0.000000,0.000000,0.000000,0.540086,NaN,NaN,NaN
2000-01-11 04:00:00,19.775000,18.150000,90.461250,101041.411625,2.916929,0.000000,0.0,1,0.000000,0.082672,0.000000,0.407318,NaN,NaN,NaN


In [ ]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

In [ ]:
gb.fit(X_train,y_train)

print('GradientBoosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))
print(metrics.mean_squared_error(y_test,gb.predict(X_test)))

GradientBoosting
0.008698542184417863
0.01558140090129325
0.013494449737284626


In [ ]:
adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))
print(metrics.mean_squared_error(y_test,adb.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Adaboost
0.0004256903104762357
0.017563474257258502
0.013934661315488123


In [ ]:
# voting ensemble
from sklearn.ensemble import VotingRegressor

estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]

vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))
print(metrics.mean_squared_error(y_test,vr.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ensemble
0.0037470677650154028
0.37827870511073536
0.38449449953055204


In [ ]:
test['pred']=  adb.predict(X_test)

In [ ]:
# with previous data 6 hour prediction
import plotly.express as px
fig = px.line(test.reset_index(),x='date_time',y=['fog_index_future','pred'])

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1,label='1y',step='year',stepmode='todate'),
            dict(count=2,label='2y',step='year',stepmode='todate'),
            dict(count=3,label='3y',step='year',stepmode='todate'),
            dict(step='all')
        ])
    )
)
fig.show()

In [ ]:
# for 1 day Model_2

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-4)




#Adding more columns
df['one_day_fog_index'] = df['fog_index'].shift(4)
df['two_day_fog_index'] = df['one_day_fog_index'].shift(4)
df['three_day_fog_index'] = df['two_day_fog_index'].shift(4)
df['four_day_fog_index'] = df['three_day_fog_index'].shift(4)
df['five_day_fog_index'] = df['four_day_fog_index'].shift(4)


df.dropna(inplace=True)

# selecting winter month(NDJF)
#train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]

#train.dropna(inplace= True)
X_train,y_train = df.drop(["fog_index_future"],axis=1),df["fog_index_future"]


# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-4)


#Adding more columns
val['one_day_fog_index'] = val['fog_index'].shift(4)
val['two_day_fog_index'] = val['one_day_fog_index'].shift(4)
val['three_day_fog_index'] = val['two_day_fog_index'].shift(4)
val['four_day_fog_index'] = val['three_day_fog_index'].shift(4)
val['five_day_fog_index'] = val['four_day_fog_index'].shift(4)



val.dropna(inplace=True)

# selecting winter month(NDJF)
#validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]

#validate.dropna(inplace=True)

X_val,y_val = val.drop(["fog_index_future"],axis=1),val["fog_index_future"]


# taking testidation dataset
test = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/test_preprocessed.csv')

test["date_time"]=pd.to_datetime(test.date_time)
test.set_index("date_time",inplace=True)

test['fog_index_future']= test['fog_index'].shift(-4)
test['one_day_fog_index'] = test['fog_index'].shift(4)

#Adding more columns
test['two_day_fog_index'] = test['one_day_fog_index'].shift(4)
test['three_day_fog_index'] = test['two_day_fog_index'].shift(4)
test['four_day_fog_index'] = test['three_day_fog_index'].shift(4)
test['five_day_fog_index'] = test['four_day_fog_index'].shift(4)

test.dropna(inplace=True)

# selecting winter month(NDJF)
#test_data =  test[(test.index.month==1) | (test.index.month==2)|(test.index.month==11) |(test.index.month==12)]

#test_data.dropna(inplace=True)
X_test,y_test = test.drop(["fog_index_future"],axis=1),test["fog_index_future"]


stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)
X_test_stand = stand_scaler.transform(X_test)
rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))
print(metrics.mean_squared_error(y_test,rf.predict(X_test)))
lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))
print(metrics.mean_squared_error(y_test,lr.predict(X_test_stand)))

RandomforestRegressor
0.0019359496988625225
0.026371513659274516
0.02360821980158764
LinearRegressor
0.01402870447579935
1.4260879740746935
1.4358609101285424


In [ ]:
df.head(7)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index,fog_index_future,one_day_fog_index,two_day_fog_index,three_day_fog_index,four_day_fog_index,five_day_fog_index
date_time,,,,,,,,,,,,,,,
2000-01-01 04:00:00,10.000000,10.000000,100.000000,101631.207878,0.193121,10693.704233,19800.0,1,0.540086,0.000000,NaN,NaN,NaN,NaN,NaN
2000-01-01 10:00:00,10.860000,10.430000,97.136333,101849.065640,0.410382,8798.058547,21600.0,1,0.407318,0.000000,NaN,NaN,NaN,NaN,NaN
2000-01-01 16:00:00,13.175000,11.625000,90.555000,101629.796892,1.425607,2349.704586,21600.0,1,0.108783,0.000000,NaN,NaN,NaN,NaN,NaN
2000-01-01 22:00:00,9.250000,9.125000,98.656250,101722.922625,0.193121,11665.859163,21600.0,1,0.540086,0.000000,NaN,NaN,NaN,NaN,NaN
2000-01-02 04:00:00,14.304545,10.927273,80.156818,101076.814791,2.980937,0.000000,0.0,1,0.000000,0.000000,0.540086,NaN,NaN,NaN,NaN
2000-01-02 10:00:00,17.990000,11.290000,65.721000,101060.036770,3.862416,0.000000,0.0,1,0.000000,0.082672,0.407318,NaN,NaN,NaN,NaN
2000-01-02 16:00:00,22.090000,12.240000,54.080000,100782.352800,3.822182,0.000000,0.0,1,0.000000,0.000000,0.108783,NaN,NaN,NaN,NaN


In [ ]:
lr.coef_

array([ 0.01516499, -0.01334216,  0.02203068, -0.00162583, -0.01558994,
       -0.01139458,  0.00431494,  0.0106891 ,  0.04197514,  0.01411834,
        0.00583622,  0.00154265,  0.00089093,  0.0120657 ])

In [ ]:
lr.intercept_

0.08263777924379571

In [ ]:
train.head(2)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index,fog_index_future,one_day_fog_index,two_day_fog_index,three_day_fog_index,four_day_fog_index,five_day_fog_index
date_time,,,,,,,,,,,,,,,
2000-01-14 10:00:00,15.558333,12.533333,82.696250,101347.597725,2.966550,906.186258,7200.0,1,0.125859,0.082739,0.084252,0.0,0.000000,0.0,0.540086
2000-01-14 16:00:00,21.916667,11.472222,53.096574,101139.240678,3.352792,160.189388,1800.0,1,0.088994,0.000000,0.000000,0.0,0.082672,0.0,0.407318


In [ ]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

In [ ]:
gb.fit(X_train,y_train)

print('GradientBoosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))
print(metrics.mean_squared_error(y_test,gb.predict(X_test)))

GradientBoosting
0.010866730045333513
0.023978593488319707
0.020476549172539944


In [ ]:
adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))
print(metrics.mean_squared_error(y_test,adb.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Adaboost
0.0007145931481913608
0.024367709829876755
0.020196017671890824


In [ ]:
# voting ensemble
from sklearn.ensemble import VotingRegressor

estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]

vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))
print(metrics.mean_squared_error(y_test,vr.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ensemble
0.0046783273512966735
0.12377940400599982
0.12487571516573676


In [ ]:
# for 3 day Model_2

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-12)
df['one_day_fog_index'] = df['fog_index'].shift(4)

#Adding more columns
df['two_day_fog_index'] = df['one_day_fog_index'].shift(4)
df['three_day_fog_index'] = df['two_day_fog_index'].shift(4)
df['four_day_fog_index'] = df['three_day_fog_index'].shift(4)
df['five_day_fog_index'] = df['four_day_fog_index'].shift(4)


df.dropna(inplace=True)

# selecting winter month(NDJF)
#train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]

#train.dropna(inplace= True)
X_train,y_train = df.drop(["fog_index_future"],axis=1),df["fog_index_future"]


# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-12)
val['one_day_fog_index'] = val['fog_index'].shift(4)

#Adding more columns
val['two_day_fog_index'] = val['one_day_fog_index'].shift(4)
val['three_day_fog_index'] = val['two_day_fog_index'].shift(4)
val['four_day_fog_index'] = val['three_day_fog_index'].shift(4)
val['five_day_fog_index'] = val['four_day_fog_index'].shift(4)


val.dropna(inplace=True)

# selecting winter month(NDJF)
#validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


#validate.dropna(inplace= True)
X_val,y_val = val.drop(["fog_index_future"],axis=1),val["fog_index_future"]


# taking testidation dataset
test = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/test_preprocessed.csv')

test["date_time"]=pd.to_datetime(test.date_time)
test.set_index("date_time",inplace=True)

test['fog_index_future']= test['fog_index'].shift(-12)
test['one_day_fog_index'] = test['fog_index'].shift(4)

#Adding more columns
test['two_day_fog_index'] = test['one_day_fog_index'].shift(4)
test['three_day_fog_index'] = test['two_day_fog_index'].shift(4)
test['four_day_fog_index'] = test['three_day_fog_index'].shift(4)
test['five_day_fog_index'] = test['four_day_fog_index'].shift(4)

test.dropna(inplace=True)

# selecting winter month(NDJF)
#test_data =  test[(test.index.month==1) | (test.index.month==2)|(test.index.month==11) |(test.index.month==12)]

#test_data.dropna(inplace=True)
X_test,y_test = test.drop(["fog_index_future"],axis=1),test["fog_index_future"]



stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)
X_test_stand = stand_scaler.transform(X_test)

rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))
print(metrics.mean_squared_error(y_test,rf.predict(X_test)))

lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))
print(metrics.mean_squared_error(y_test,lr.predict(X_test_stand)))

RandomforestRegressor
0.002207662936863808
0.03421897010612442
0.027930703325219113
LinearRegressor
0.0157521621595437
0.14140612555378484
0.14156241914534484


In [ ]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

In [ ]:
gb.fit(X_train,y_train)

print('GradientBoosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))
print(metrics.mean_squared_error(y_test,gb.predict(X_test)))

GradientBoosting
0.012495903550883519
0.028619041037247422
0.02078871818130796


In [ ]:
adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))
print(metrics.mean_squared_error(y_test,adb.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Adaboost
0.000823152356472211
0.032413108682082185
0.022303213361397258


In [ ]:
# voting ensemble
from sklearn.ensemble import VotingRegressor

estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]

vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))
print(metrics.mean_squared_error(y_test,vr.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ensemble
0.005375008502514843
0.03755207160371063
0.031905506342396174


In [ ]:
# for 5 day Model_2

df = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/train_preprocessed.csv')

df["date_time"]=pd.to_datetime(df.date_time)
df.set_index("date_time",inplace=True)

df['fog_index_future']= df['fog_index'].shift(-20)
df['one_day_fog_index'] = df['fog_index'].shift(4)

#Adding more columns
df['two_day_fog_index'] = df['one_day_fog_index'].shift(4)
df['three_day_fog_index'] = df['two_day_fog_index'].shift(4)
df['four_day_fog_index'] = df['three_day_fog_index'].shift(4)
df['five_day_fog_index'] = df['four_day_fog_index'].shift(4)


df.dropna(inplace=True)

# selecting winter month(NDJF)
#train =  df[(df.index.month==1) | (df.index.month==2)|(df.index.month==11) |(df.index.month==12)]

#train.dropna(inplace= True)
X_train,y_train = df.drop(["fog_index_future"],axis=1),df["fog_index_future"]


# taking validation dataset
val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/val_preprocessed.csv')

val["date_time"]=pd.to_datetime(val.date_time)
val.set_index("date_time",inplace=True)

val['fog_index_future']= val['fog_index'].shift(-20)
val['one_day_fog_index'] = val['fog_index'].shift(4)

#Adding more columns
val['two_day_fog_index'] = val['one_day_fog_index'].shift(4)
val['three_day_fog_index'] = val['two_day_fog_index'].shift(4)
val['four_day_fog_index'] = val['three_day_fog_index'].shift(4)
val['five_day_fog_index'] = val['four_day_fog_index'].shift(4)



val.dropna(inplace=True)

# selecting winter month(NDJF)
#validate =  val[(val.index.month==1) | (val.index.month==2)|(val.index.month==11) |(val.index.month==12)]


#validate.dropna(inplace= True)
X_val,y_val = val.drop(["fog_index_future"],axis=1),val["fog_index_future"]


# taking testidation dataset
test = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Model_1/test_preprocessed.csv')

test["date_time"]=pd.to_datetime(test.date_time)
test.set_index("date_time",inplace=True)

test['fog_index_future']= test['fog_index'].shift(-20)
test['one_day_fog_index'] = test['fog_index'].shift(4)

#Adding more columns
test['two_day_fog_index'] = test['one_day_fog_index'].shift(4)
test['three_day_fog_index'] = test['two_day_fog_index'].shift(4)
test['four_day_fog_index'] = test['three_day_fog_index'].shift(4)
test['five_day_fog_index'] = test['four_day_fog_index'].shift(4)
test.dropna(inplace=True)

# selecting winter month(NDJF)
#test_data =  test[(test.index.month==1) | (test.index.month==2)|(test.index.month==11) |(test.index.month==12)]

#test_data.dropna(inplace=True)
X_test,y_test = test.drop(["fog_index_future"],axis=1),test["fog_index_future"]


stand_scaler = StandardScaler()
X_train_stand = stand_scaler.fit_transform(X_train)
X_val_stand = stand_scaler.transform(X_val)
X_test_stand = stand_scaler.transform(X_test)

rf = RandomForestRegressor()


rf.fit(X_train,y_train)

print('RandomforestRegressor')
print(metrics.mean_squared_error(y_train,rf.predict(X_train)))
print(metrics.mean_squared_error(y_val,rf.predict(X_val)))
print(metrics.mean_squared_error(y_test,rf.predict(X_test)))

lr = LinearRegression()


lr.fit(X_train_stand,y_train)

print('LinearRegressor')
print(metrics.mean_squared_error(y_train,lr.predict(X_train_stand)))
print(metrics.mean_squared_error(y_val,lr.predict(X_val_stand)))
print(metrics.mean_squared_error(y_test,lr.predict(X_test_stand)))

RandomforestRegressor
0.0023719808407854023
0.03928915948561819
0.037562401535923134
LinearRegressor
0.016840241503851126
0.20451295212460266
0.2068855998865074


In [ ]:
gb =  GradientBoostingRegressor()
adb = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='friedman_mse',max_depth=15))

In [ ]:
gb.fit(X_train,y_train)

print('GradientBoosting')
print(metrics.mean_squared_error(y_train,gb.predict(X_train)))
print(metrics.mean_squared_error(y_val,gb.predict(X_val)))
print(metrics.mean_squared_error(y_test,gb.predict(X_test)))

GradientBoosting
0.013707942962626504
0.08754809434854983
0.0970773893971096


In [ ]:
adb.fit(X_train,y_train)

print('Adaboost')
print(metrics.mean_squared_error(y_train,adb.predict(X_train)))
print(metrics.mean_squared_error(y_val,adb.predict(X_val)))
print(metrics.mean_squared_error(y_test,adb.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Adaboost
0.0010118526610480261
0.036666447349562266
0.029117711120901852


In [ ]:
# voting ensemble
from sklearn.ensemble import VotingRegressor

estimator = [('lr',lr),('rf',rf),('gb',gb),('adb',adb)]

vr = VotingRegressor(estimator)

vr.fit(X_train,y_train)

print('ensemble')
print(metrics.mean_squared_error(y_train,vr.predict(X_train)))
print(metrics.mean_squared_error(y_val,vr.predict(X_val)))
print(metrics.mean_squared_error(y_test,vr.predict(X_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


ensemble
0.0058299661593907304
0.06040258695924927
0.0610191911447739
